In [1]:
%matplotlib inline
    
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

from sklearn import model_selection
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [3]:
# final df for model building
df1 = pd.read_csv("ML_model_wk40_to_20.csv")
df1.dropna(how='any', inplace=True)
del df1['ILI_weeks']
del df1['Unnamed: 0']
#del df1['week']
df1.head(2)

Year  week  a_influenza  acute_bronchitis  body_temperature  \
0  2009    40           44                33                81   
1  2009    41           51                51                77   

   braun_thermoscan  break_a_fever  bronchitis  chest_cold  cold_and_flu  \
0                69             22          40          35            36   
1                46             24          43          35            43   

      ...      walking_pneumonia  what_to_do_if_you_have_the_flu  \
0     ...                     47                              56   
1     ...                     53                              58   

   Flu_Visit_Count  ILI_Visit_Count  Unspecified  CDC_Unweighted_ILI  \
0          0.01338          0.01763      0.03074             5.66087   
1          0.01620          0.02103      0.03554             6.81522   

   ILI_lagwk1  ILI_lagwk2  ILI_lagwk3  ILI_lagwk4  
0     6.81522     7.61889     7.38836     6.33927  
1     7.61889     7.38836     6.33927     4.94349  

[2 rows x 139 columns]

In [4]:
#  combining year and week column 
def combine_year_week(row):
    return int(row["Year"]) * 100 + int(row["week"])

In [5]:
df1["YearWeek"] = df1.apply(combine_year_week, axis=1)

In [6]:
# reset index to YearWeek
df1.index = df1["YearWeek"]

In [7]:
model_df = df1.drop(["Year", "week", "YearWeek"], axis=1)
model_df.head(2)

a_influenza  acute_bronchitis  body_temperature  braun_thermoscan  \
YearWeek                                                                      
200940             44                33                81                69   
200941             51                51                77                46   

          break_a_fever  bronchitis  chest_cold  cold_and_flu  cold_or_flu  \
YearWeek                                                                     
200940               22          40          35            36           37   
200941               24          43          35            43           49   

          cold_versus_flu     ...      walking_pneumonia  \
YearWeek                      ...                          
200940                 30     ...                     47   
200941                 41     ...                     53   

          what_to_do_if_you_have_the_flu  Flu_Visit_Count  ILI_Visit_Count  \
YearWeek                                                                     
200940                                56          0.01338          0.01763   
200941                                58          0.01620          0.02103   

          Unspecified  CDC_Unweighted_ILI  ILI_lagwk1  ILI_lagwk2  ILI_lagwk3  \
YearWeek                                                                        
200940        0.03074             5.66087     6.81522     7.61889     7.38836   
200941        0.03554             6.81522     7.61889     7.38836     6.33927   

          ILI_lagwk4  
YearWeek              
200940       6.33927  
200941       4.94349  

[2 rows x 137 columns]

# Independent variables
        # 3 independent variables from athena EHR 
            # [(flu visit counts)/ (total patient visit counts) 
            # (ILI visit counts)/ (total patient visit counts)
            # (unspecified viral or ILI visit counts)/ (total patient visit counts)]
        # CDC historical CDC_Unweighted_ILI values: collected from 2009 to 2016 (week 40 to 20)
        # 129 google search terms related to flu
        
      ======> 3 + 1 + 129
      
# Dependent variables 
        # 4 ILI weeks offset by 1 week

# split df to train dataset, dataset before year2015 and week 40 used at training dataset

In [8]:
train = model_df[model_df.index < 201540]
# train.head()


# defining  targets/labels to  y axis
                y_train = ILI lag week 1

                yy_train = ILI lag week 2

                yyy_train = ILI lag week 3

                yyyy_train = ILI lag week 4

In [9]:
y_train = train["ILI_lagwk1"]
# y_train

In [10]:
yy_train = train["ILI_lagwk2"]

In [11]:
yyy_train = train["ILI_lagwk3"]

In [12]:
yyyy_train =train["ILI_lagwk4"]

In [13]:
X_train = train.drop(["ILI_lagwk1", "ILI_lagwk2", "ILI_lagwk3", "ILI_lagwk4"], axis=1)
# X_train

# split df to train dataset,dataset after year2015 and week 40 used at training dataset

In [14]:
test = model_df[model_df.index >= 201540]
# test.head()

In [15]:
y_test = test["ILI_lagwk1"]

In [16]:
yy_test = test["ILI_lagwk2"]

In [17]:
yyy_test = test["ILI_lagwk3"]

In [18]:
yyyy_test = test["ILI_lagwk4"]

In [19]:
X_test = test.drop(["ILI_lagwk1", "ILI_lagwk2", "ILI_lagwk3", "ILI_lagwk4"], axis=1)

# Reference https://shankarmsy.github.io/stories/gbrt-sklearn.html#

##https://www.youtube.com/watch?v=IXZKgIsZRm0

# building a model for y_test = test["ILI_lagwk1"]

In [20]:
model1 = SVR()

In [ ]:
# StandardScaler to transform features
scaler = StandardScaler()  
scaler.fit(X_train) 

X_train_scaled = scaler.transform(X_train) 
X_test_scaled = scaler.transform(X_test)

In [21]:
model1.fit(X_train, y_train)

C:\Users\emame\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

In [22]:
y_pred = model1.predict(X_test)

In [23]:
r2_score(y_test, y_pred)

-0.010715330711997373

In [24]:
mean_absolute_error(y_test, y_pred)

0.48387093649612606

In [25]:
print("R-squared for ILI_lagwk1 Train: %.2f" %model1.score(X_train, y_train)) 
print("R-squared for ILI_lagwk1 Test : %.2f" %model1.score(X_test, y_test)) 

R-squared for ILI_lagwk1 Train: 0.61
R-squared for ILI_lagwk1 Test : -0.01


In [26]:
predictions1 = model1.predict(X_test) 

In [27]:
# predict the values of y
y_test_unraveled = y_test.values.ravel()
y_test.index

Int64Index([201540, 201541, 201542, 201543, 201544, 201545, 201546, 201547,
            201548, 201549, 201550, 201551, 201552, 201601, 201602, 201603,
            201604, 201605, 201606, 201607, 201608, 201609, 201610, 201611,
            201612, 201613, 201614, 201615, 201616, 201617, 201618, 201619,
            201620],
           dtype='int64', name='YearWeek')

In [28]:
# store prediction and actual to df
pred_df = pd.DataFrame({"Prediction_ILI_lagwk1": predictions1, "Actual": y_test_unraveled}).reset_index(drop=True)
pred_df.index = y_test.index
pred_df.head()

Prediction_ILI_lagwk1   Actual
YearWeek                                
201540                 2.055565  1.33029
201541                 2.055566  1.41368
201542                 2.055566  1.36882
201543                 2.055565  1.48309
201544                 2.055566  1.54494

# # Create the GridSearchCV model

In [29]:
# from sklearn.model_selection import GridSearchCV
# param_grid = {'learning_rate':[ 0.02, 0.01, 0.1, 0.05], 
#             'max_depth':[1, 3, 5 ,7], 
#             'min_samples_leaf':[3, 5, 7 ,9], 
# #             'max_features':[0.1,0.3,1.0],
#             'n_estimators': [300, 500, 1000, 2000]
#              } 

# est = GradientBoostingRegressor()
# gs_cv = GridSearchCV(est, param_grid).fit(X_train, y_train)

# # best hyperparameter setting

# gs_cv.best_est 

# selecting important features  ["ILI_lagwk1"]

In [ ]:
# # GBRF predictors variable importance
# # store most important variables under importances
# importances = gbrt.feature_importances_

In [ ]:
# gbrt.feature_importances_.sum()

In [ ]:
# # store most important variables/features under importances
# importances = gbrt.feature_importances_
# # importances

In [ ]:
# sorted_features = sorted(zip(importances, X_train.columns), reverse=True)
# # sorted_features

In [ ]:
# # sort important varibles and sotre them under indices
# important_columns = []
# for fimportance, name in sorted(zip(importances, X_train.columns), reverse=True):
#     if fimportance > 0.00009:
#         important_columns.append(name)

In [ ]:
# important_feature_columns = important_columns
# # important_feature_columns

In [ ]:
# important_features_df = model_df[important_feature_columns]
# important_features_df.head()

# defining important feature X train and X test dataset

In [ ]:
# X_trainim = important_features_df[important_features_df.index < 201540]
# # trainim
# # .head(2)

In [ ]:
# X_testim = important_features_df[important_features_df.index >= 201540]
# # testim
# # .head()

In [ ]:
# gbrtim_ili_lag1 = GradientBoostingRegressor(n_estimators = 500, max_depth = 5) # number of sequential trees to be modeled

In [ ]:
# gbrtim_ili_lag1.fit(X_trainim, y_train) 

In [ ]:
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
# r2_score(y_test, y_pred)

In [ ]:
# mean_absolute_error(y_test, y_pred)

In [ ]:
# # GBR with important features for ili_lag1 week 1
# print("R-squared for Train gbrtim_ili_lag1: %.2f" %gbrtim_ili_lag1.score(X_trainim, y_train)) 
# print("R-squared for Test gbrtim_ili_lag1: %.2f" %gbrtim_ili_lag1.score(X_testim, y_test)) 

In [ ]:
# # predict the values of y with important feature extraction for ili week 1
# predictions = gbrtim_ili_lag1.predict(X_testim)
# y_test_unraveled = y_test.values.ravel()
# y_test.index

In [ ]:
# Make predictions using the X_test and y_test data
# Print at least 10 predictions vs their actual labels
# predictions = gbrtim_ili_lag1.predict(X_testim)
# print(f"First 10 Predictions: {predictions[:10]}")
# print(f"First 10 Actual labels: {y_test_unraveled[:10]}")

In [ ]:
# # Print predictions vs their actual labels
# pred_df = pd.DataFrame({"Prediction": predictions, "Actual": y_test_unraveled}).reset_index(drop=True)
# pred_df.index = y_test.index
# pred_df.head()

# building a model for y_test = test["ILI_lagwk2"]

In [ ]:
model2 = SVR()

In [ ]:
model2.fit(X_train, yy_train) 

In [ ]:
yy_pred = model2.predict(X_test)

In [ ]:
r2_score(yy_test, yy_pred)

In [ ]:
mean_absolute_error(yy_test, yy_pred)

In [ ]:
print("R-squared for ILI_lagwk2 Train: %.2f" %model2.score(X_train, yy_train)) 
print("R-squared for ILI_lagwk2 Test: %.2f" %model2.score(X_test, yy_test)) 

In [ ]:
predictions2 = model2.predict(X_test) 

In [ ]:
# predict the values of y
yy_test_unraveled = yy_test.values.ravel()
yy_test.index

In [ ]:
# store prediction and actual to df
pred_df = pd.DataFrame({"Prediction_ILI_lagwk2": predictions2, "Actual": yy_test_unraveled}).reset_index(drop=True)
pred_df.index = yy_test.index
pred_df.head()

In [ ]:
# # Create the GridSearchCV model

# from sklearn.model_selection import GridSearchCV
# param_grid = {'learning_rate':[ 0.02, 0.01, 0.1, 0.05], 
#             'max_depth':[1, 3, 5 ,7], 
#             'min_samples_leaf':[3, 5, 7 ,9], 
# #             'max_features':[0.1,0.3,1.0],
#             'n_estimators': [300, 500, 1000, 2000]
#              } 

# est = GradientBoostingRegressor()
# gs_cv = GridSearchCV(est, param_grid).fit(X_train, y_train)

# # best hyperparameter setting

# gs_cv.best_est 

# selecting important features  ["ILI_lagwk2"]

# defining important feature X train and X test dataset

# building a model for y_test = test["ILI_lagwk3"]

In [ ]:
model3 = SVR()

In [ ]:
model3.fit(X_train, yyy_train) 

In [ ]:
yyy_pred = model3.predict(X_test)

In [ ]:
r2_score(yyy_test, yyy_pred)

In [ ]:
mean_absolute_error(yyy_test, yyy_pred)

In [ ]:
print("R-squared for Train: %.2f" %model3.score(X_train, yyy_train)) 
print("R-squared for Test: %.2f" %model3.score(X_test, yyy_test)) 

In [ ]:
predictions3 = gbrt3.predict(X_test) 

In [ ]:
# predict the values of y
predictions = model3.predict(X_test)
yyy_test_unraveled = yyy_test.values.ravel()
yyy_test.index

In [ ]:
pred_df = pd.DataFrame({"Prediction_ILI_lagwk3": predictions3, "Actual": yyy_test_unraveled}).reset_index(drop=True)
pred_df.index = yyy_test.index
pred_df.head()

# building a model for y_test = test["ILI_lagwk4"]

In [ ]:
model4 = SVR()

In [ ]:
model4.fit(X_train, yyyy_train) 

In [ ]:
yyyy_pred = model4.predict(X_test)

In [ ]:
r2_score(yyyy_test, yyyy_pred)

In [ ]:
mean_absolute_error(yyyy_test, yyyy_pred)

In [ ]:
print("R-squared for Train: %.2f" %model4.score(X_train, yyyy_train)) 
print("R-squared for Test: %.2f" %model4.score(X_test, yyyy_test)) 

In [ ]:
# predict the values of y
predictions4 = model4.predict(X_test)
yyyy_test_unraveled = yyyy_test.values.ravel()
yyyy_test.index

In [ ]:
# store prediction and actual to df
pred_df = pd.DataFrame({"Prediction_ILI_lagwk1": predictions4, "Actual": yyyy_test_unraveled}).reset_index(drop=True)
pred_df.index = yyyy_test.index
pred_df.head()